### Import packages

In [1]:
import numpy as np
import pandas as pd

np.random.seed(1337)  # for reproducibility
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.regression import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from dbn.tensorflow import SupervisedDBNRegression

### Defining Road and Year for dataset

In [2]:
# Read the dataset
ROAD = "Taft Ave."
YEAR = "2015"
EXT = ".csv"

### Defining Hyper parameters

In [3]:
RBM_EPOCHS = 5
DBN_EPOCHS = 150
RBM_LEARNING_RATE = 0.01
DBN_LEARNING_RATE = 0.01
HIDDEN_LAYER_STRUCT = [20, 50, 100]
ACTIVE_FUNC = 'relu'
BATCH_SIZE = 16

### Preparing PM1 and PM2 Output dataset for training

In [15]:
traffic_data = pd.read_csv("data/mmda/eng_win2_mmda_" + ROAD + "_" + YEAR + EXT, skipinitialspace=True)

In [4]:
pm1_results = pd.read_csv("output/pm1_output_" + ROAD + "_" + YEAR + EXT, skipinitialspace=True)
pm2_results = pd.read_csv("output/pm2_output_" + ROAD + "_" + YEAR + EXT, skipinitialspace=True)

In [5]:
d = {'PM1-Output': pm1_results.Predicted, 'PM2-Output': pm2_results.Predicted}
fusion_dataset = pd.DataFrame(data=d)
fusion_dataset = np.array(fusion_dataset)
actual_dataset = pm1_results.Actual

In [6]:
# To-be Predicted variable
Y = actual_dataset
Y = Y.round(5)

# Other data
X = fusion_dataset

# Splitting data
X_train = X
Y_train = Y
X_test = X_train
Y_test = Y_train
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

# Data scaling
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)

### Training the Fusion Center

In [7]:
# Training
fc = SupervisedDBNRegression(hidden_layers_structure=HIDDEN_LAYER_STRUCT,
                                    learning_rate_rbm=RBM_LEARNING_RATE,
                                    learning_rate=DBN_LEARNING_RATE,
                                    n_epochs_rbm=RBM_EPOCHS,
                                    n_iter_backprop=DBN_EPOCHS,
                                    batch_size=BATCH_SIZE,
                                    activation_function=ACTIVE_FUNC)
fc.fit(X_train, Y_train)

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 0.048787
>> Epoch 2 finished 	RBM Reconstruction error 0.044702
>> Epoch 3 finished 	RBM Reconstruction error 0.040769
>> Epoch 4 finished 	RBM Reconstruction error 0.036798
>> Epoch 5 finished 	RBM Reconstruction error 0.036308
>> Epoch 1 finished 	RBM Reconstruction error 0.017163
>> Epoch 2 finished 	RBM Reconstruction error 0.016946
>> Epoch 3 finished 	RBM Reconstruction error 0.016783
>> Epoch 4 finished 	RBM Reconstruction error 0.016863
>> Epoch 5 finished 	RBM Reconstruction error 0.016335
>> Epoch 1 finished 	RBM Reconstruction error 0.001140
>> Epoch 2 finished 	RBM Reconstruction error 0.001227
>> Epoch 3 finished 	RBM Reconstruction error 0.001123
>> Epoch 4 finished 	RBM Reconstruction error 0.001132
>> Epoch 5 finished 	RBM Reconstruction error 0.001182
[END] Pre-training step
[START] Fine tuning step:
>> Epoch 0 finished 	ANN training loss 0.035447
>> Epoch 1 finished 	ANN training loss 0.035206
>>

>> Epoch 149 finished 	ANN training loss 0.000006
[END] Fine tuning step


SupervisedDBNRegression(batch_size=16, dropout_p=0, l2_regularization=1.0,
            learning_rate=0.01, n_iter_backprop=150, verbose=True)

### Testing the Fusion Center

In [8]:
# Test
X_test = min_max_scaler.transform(X_test)
Y_pred = fc.predict(X_test)
print('Done.\nR-squared: %f\nMSE: %f' % (r2_score(Y_test, Y_pred), mean_squared_error(Y_test, Y_pred)))
print('MAE: %f' %(mean_absolute_error(Y_test, Y_pred)))

Done.
R-squared: 0.999823
MSE: 0.000006
MAE: 0.001904


### Exporting predicted and actual results

In [27]:
startIndex = traffic_data.shape[0] - fc_results.shape[0]
dt = traffic_data.dt[startIndex:,]
temp = []
for i in range(len(Y_pred)):
    temp.append(Y_pred[i][0])
d = {'Predicted': temp, 'Actual': Y_test, 'dt':dt}
fc_results = pd.DataFrame(data=d)

In [28]:
fc_results

Actual  Predicted                dt
11563  0.34164   0.342362  01/05/2015 10:45
11564  0.33944   0.340204  01/05/2015 11:00
11565  0.33737   0.338181  01/05/2015 11:15
11566  0.33543   0.336288  01/05/2015 11:30
11567  0.33362   0.334522  01/05/2015 11:45
11568  0.33193   0.332879  01/05/2015 12:00
11569  0.33037   0.331339  01/05/2015 12:15
11570  0.32892   0.329909  01/05/2015 12:30
11571  0.32758   0.328574  01/05/2015 12:45
11572  0.32636   0.327350  01/05/2015 13:00
11573  0.32524   0.326232  01/05/2015 13:15
11574  0.32423   0.325224  01/05/2015 13:30
11575  0.32331   0.324319  01/05/2015 13:45
11576  0.32250   0.323511  01/05/2015 14:00
11577  0.32178   0.322793  01/05/2015 14:15
11578  0.32114   0.322170  01/05/2015 14:30
11579  0.32060   0.321632  01/05/2015 14:45
11580  0.32014   0.321177  01/05/2015 15:00
11581  0.31976   0.320803  01/05/2015 15:15
11582  0.31946   0.320505  01/05/2015 15:30
11583  0.31923   0.320280  01/05/2015 15:45
11584  0.31907   0.320125  01/05/2015 16:00
11585  0.31898   0.320038  01/05/2015 16:15
11586  0.31895   0.320006  01/05/2015 16:30
11587  0.31898   0.320035  01/05/2015 16:45
11588  0.31907   0.320121  01/05/2015 17:00
11589  0.31921   0.320260  01/05/2015 17:15
11590  0.31940   0.320424  01/05/2015 17:30
11591  0.31964   0.320706  01/05/2015 17:45
11592  0.31992   0.321043  01/05/2015 18:00
...        ...        ...               ...
35010  0.38542   0.384911  31/12/2015 16:30
35011  0.38541   0.384861  31/12/2015 16:45
35012  0.38540   0.384810  31/12/2015 17:00
35013  0.38539   0.384755  31/12/2015 17:15
35014  0.38538   0.384718  31/12/2015 17:30
35015  0.38537   0.384680  31/12/2015 17:45
35016  0.38536   0.384642  31/12/2015 18:00
35017  0.38535   0.384605  31/12/2015 18:15
35018  0.38534   0.384580  31/12/2015 18:30
35019  0.38533   0.384556  31/12/2015 18:45
35020  0.38532   0.384531  31/12/2015 19:00
35021  0.38531   0.384505  31/12/2015 19:15
35022  0.38530   0.384499  31/12/2015 19:30
35023  0.38529   0.384492  31/12/2015 19:45
35024  0.38528   0.384479  31/12/2015 20:00
35025  0.38527   0.384467  31/12/2015 20:15
35026  0.38526   0.384447  31/12/2015 20:30
35027  0.38526   0.384427  31/12/2015 20:45
35028  0.38525   0.384407  31/12/2015 21:00
35029  0.38524   0.384388  31/12/2015 21:15
35030  0.38523   0.384378  31/12/2015 21:30
35031  0.38522   0.384369  31/12/2015 21:45
35032  0.38521   0.384360  31/12/2015 22:00
35033  0.38521   0.384351  31/12/2015 22:15
35034  0.38520   0.384352  31/12/2015 22:30
35035  0.38519   0.384352  31/12/2015 22:45
35036  0.38518   0.384353  31/12/2015 23:00
35037  0.38517   0.384354  31/12/2015 23:15
35038  0.38517   0.384362  31/12/2015 23:30
35039  0.38516   0.384369  31/12/2015 23:45

[23477 rows x 3 columns]

In [ ]:
fc_results.to_csv("output/fc_output_" + ROAD  + "_" + YEAR + EXT, encoding='utf-8', index=False)

### Saving the model

In [ ]:
# Save the model
fc.save('models/fc_' + ROAD + '_' + YEAR + '.pkl')